# Import Required Packages

In [1]:
# Imports
import os
import datetime
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time



# Input data from User

In [2]:
#Market analysed: 'Investment','FullYear','DayAhead','Balancing' (choose one or several)
markets=['DayAhead'] 
output='PriceElectricityHourly'
first_timestep="2012-01-02"
#Meaning of SSS and TTT in the data: 'DaysHours','Hours5min','WeeksHours'
meaning_SSS_TTT='DaysHours'
#Time size of each time step in TTT for creating timestamp
size_timestep="3600s"


# Plot Settings

In [3]:
# Set plotting specifications
%matplotlib inline
plt.rcParams.update({'font.size': 21})
plt.rcParams['xtick.major.pad']='12'
plt.rc('legend', fontsize=16)
y_limit = 1.1
lw = 3

# Read Input Files

In [4]:
data=pd.DataFrame()
for market in markets:
    csvfiles = []
    for file in glob.glob("./input/results/" + market + "/*.csv"):
        csvfiles.append(file)

    csvfiles=[file.replace('./input\\','') for file in csvfiles] 
    csvfiles=[file.replace('.csv','') for file in csvfiles]  
    csvfiles=[file.split('_') for file in csvfiles]  
    csvfiles = np.asarray(csvfiles)  
    csvfiles=pd.DataFrame.from_records(csvfiles)
    
    csvfiles.rename(columns={0: 'Output', 1: 'Scenario',2: 'Year',3:'Subset'}, inplace=True)
    scenarios=csvfiles.Scenario.unique().tolist()
    years=csvfiles.Year.unique().tolist()
    subsets=csvfiles.Subset.unique().tolist()

    for scenario in scenarios:
        for year in years:
            for subset in subsets:
                file = "./input/results/"+ market + "/"+ output + "_" + scenario + "_" + year + "_" + subset + ".csv"
                if os.path.isfile(file):
                    df=pd.read_csv(file,encoding='utf8')
                    df['Scenario'] = scenario
                    df['Market']   = market
                    #Renaming columns just in case timeconversion was required
                    df.rename(columns = {'G':'GGG', 'C':'CCC', 'Y':'YYY','TTT_NEW':'TTT','SSS_NEW':'SSS'}, inplace = True) 
                    data=data.append(df) 

                
                

In [5]:
#Timestamp addition
full_timesteps = pd.read_csv('./input/full_timesteps_'+meaning_SSS_TTT+'.csv')
full_timesteps.Key=full_timesteps['SSS']+full_timesteps['TTT']
number_periods=len(full_timesteps.Key.unique())
full_timesteps['timestamp']= pd.date_range(first_timestep, periods = number_periods, freq =size_timestep)
dict_timestamp=dict(zip(full_timesteps.Key, full_timesteps.timestamp))
data['timestamp']=data['SSS']+data['TTT']
data['timestamp']=data['timestamp'].map(dict_timestamp)



C:\Users\s151529\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


# Additional set declaration

In [6]:
ccc = list(data.CCC.unique())
rrr = list(data.RRR.unique())
#tech_type = list(data.TECH_TYPE.unique())
#commodity = list(data.COMMODITY.unique())
#fff = list(data.FFF.unique())
sss = list(full_timesteps.SSS.unique())
ttt = list(full_timesteps.TTT.unique())


# Time step selection

In [7]:
# Seasons to investigate
# season_names = ['S01', 'S07', 'S20', 'S24', 'S28', 'S38', 'S42', 'S43']
# Make a list of every nth element of sss (1 <= nth <= number of elements in sss)
nth = 1
s = sss[0::nth]
 # Or select seasons by names
# s = season_names


In [8]:
# Terms to investigate
# term_names = ['T005', 'T019', 'T033', 'T047', 'T061', 'T075', 'T089', 'T103', 'T117', 'T131', 'T145', 'T159']
# Make a list of every nth element of ttt (1 <= nth <= number of elements in ttt)
nth = 1
t = ttt[0::nth]
# Or select terms by name
# t = term_names

# Make Directories


In [9]:
# Make output folder
if not os.path.isdir('output'):
    os.makedirs('output')

In [10]:
# Make CurtailmentHourly folder
if not os.path.isdir('output/' + output):
    os.makedirs('output/' + output)

In [11]:
# Make market folder
for market in markets:
    if not os.path.isdir('output/' + output + '/'+ market +'/Country_wise'):
        os.makedirs('output/' + output + '/'+ market +'/Country_wise')
# Make country folder
    if not os.path.isdir('output/' + output  + '/'+ market +'/Country_wise'):
        os.makedirs('output/' + output  + '/'+ market  +'/Country_wise')
    # Make country wise folders
    for c in ccc:
        if not os.path.isdir('output/' + output  + '/'+ market +'/Country_wise/' + c):
            os.makedirs('output/' + output  + '/'+ market +'/Country_wise/' + c)

# Plotting

In [12]:
# Make data frames to plot
data_plot = data[(data.SSS.isin(s)) & (data.TTT.isin(t))]


##  Data export

In [13]:
table = pd.pivot_table(data, values='Val', index=['Market','Scenario','YYY','timestamp'],
...                     columns=['RRR'], aggfunc=np.sum, fill_value=0).reset_index()

In [14]:
for market in markets:
    for scenario in scenarios:
        for year in years:
                table_export=table[table.columns.difference(['Market','Scenario','YYY'])].loc[(table['Market'] == market) & (table['Scenario'] == scenario) &  (table['YYY'].astype(str) == year)]
                table_export=table_export.set_index(['timestamp'])
                table_export.to_csv('Output/'+output+'/'+output+'_'+market+'_'+scenario+'_'+year+'.csv')